In [1]:
# Dependencies

from typing import List, Optional, Dict
from src.tree.step import Stepper, ctx, Hash, Tree
from src.mermaid import Mermaid

## Fibonacci

### Without Dynamic Programming

In [2]:
def fib(n: int, parent_id: Optional[Hash] = None):
    id = ctx.get().hash()
    ctx.get().called_with(id, parent_id, fib, [n])
    if n <= 1:
        ctx.get().returned_with(id, parent_id, n)
        return n

    r1 = fib(n - 1, id)
    r2 = fib(n - 2, id)
    ctx.get().returned_with(id, parent_id, r1 + r2)
    return r1 + r2

result, token = Stepper.run(fib, 8)
assert result == 21

tree = Tree.build(ctx.get().called_with_output_values)

ctx.reset(token)
tree.render(tree.root).to_image()

## With Dynamic Programming

In [3]:
memo: Dict[int, int] = {}

def fib(n: int, parent_id: Optional[Hash] = None):
    id = ctx.get().hash()
    ctx.get().called_with(id, parent_id, fib, [n])

    if n in memo:
        ctx.get().cached_with(id, parent_id, memo[n])
        return memo[n]

    if n <= 1:
        memo[n] = n
        ctx.get().returned_with(id, parent_id, n)
        return n

    r1 = fib(n - 1, id)
    r2 = fib(n - 2, id)
    ctx.get().returned_with(id, parent_id, r1 + r2)
    memo[n] = r1 + r2
    return r1 + r2

result, token = Stepper.run(fib, 8)
assert result == 21

tree = Tree.build(ctx.get().called_with_output_values)
ctx.reset(token)
tree.render(tree.root).to_image()

# Grid Traveler

## Without Dynamic Programming

In [4]:
def grid_traveler(m: int, n: int, parent_id: Optional[Hash] = None):
    id = ctx.get().hash()
    ctx.get().called_with(id, parent_id, grid_traveler, [m, n])

    if m == 1 and n == 1:
        ctx.get().returned_with(id, parent_id, 1)
        return 1
    
    if m == 0 or n == 0:
        ctx.get().returned_with(id, parent_id, 0)
        return 0
    
    r1 = grid_traveler(m - 1, n, id)
    r2 = grid_traveler(m, n - 1, id)
    ctx.get().returned_with(id, parent_id, r1 + r2)
    return r1 + r2

result, token = Stepper.run(grid_traveler, 3, 3)
assert result == 6

tree = Tree.build(ctx.get().called_with_output_values)
ctx.reset(token)
tree.render(tree.root).to_image()

## With Dynamic Programming

In [5]:
memo: Dict[str, int] = {}

def grid_traveler(m: int, n: int, parent_id: Optional[Hash] = None):
    id = ctx.get().hash()
    ctx.get().called_with(id, parent_id, grid_traveler, [m, n])

    key = f"{min(m, n)}-{max(m, n)}"
    if key in memo:
        ctx.get().cached_with(id, parent_id, memo[key])
        return memo[key]

    if m == 1 and n == 1:
        ctx.get().returned_with(id, parent_id, 1)
        return 1
    
    if m == 0 or n == 0:
        ctx.get().returned_with(id, parent_id, 0)
        return 0
    
    r1 = grid_traveler(m - 1, n, id)
    r2 = grid_traveler(m, n - 1, id)
    memo[key] = r1 + r2
    ctx.get().returned_with(id, parent_id, r1 + r2)
    return memo[key]

result, token = Stepper.run(grid_traveler, 3, 3)
assert result == 6

tree = Tree.build(ctx.get().called_with_output_values)
ctx.reset(token)
tree.render(tree.root).to_image()

## Can Sum

### Without Dynamic Programming

In [6]:
def can_sum(target: int, numbers: List[int], parent_id: Optional[Hash] = None) -> bool:
    id = ctx.get().hash()
    ctx.get().called_with(id, parent_id, can_sum, [target])

    if target == 0:
        ctx.get().returned_with(id, parent_id, True)
        return True
    
    if target < 0:
        ctx.get().returned_with(id, parent_id, False)
        return False

    for n in numbers:
        if can_sum(target - n, numbers, id):
            ctx.get().returned_with(id, parent_id, True)
            return True

    ctx.get().returned_with(id, parent_id, False)
    return False

result, token = Stepper.run(can_sum,  31, [7, 14])
assert result is False

tree = Tree.build(ctx.get().called_with_output_values)
ctx.reset(token)
tree.render(tree.root).to_image()

## With Dynamic Programming

In [7]:
memo: Dict[int, bool] = {}

def can_sum(target: int, numbers: List[int], parent_id: Optional[Hash] = None) -> bool:
    id = ctx.get().hash()
    ctx.get().called_with(id, parent_id, can_sum, [target])

    if target in memo:
        ctx.get().cached_with(id, parent_id, memo[target])
        return memo[target]

    if target == 0:
        ctx.get().returned_with(id, parent_id, True)
        return True
    
    if target < 0:
        ctx.get().returned_with(id, parent_id, False)
        return False

    for n in numbers:
        memo[target] = can_sum(target - n, numbers, id)
        if memo[target]:
            ctx.get().returned_with(id, parent_id, True)
            return True

    ctx.get().returned_with(id, parent_id, False)
    memo[target] = False
    return False

result, token = Stepper.run(can_sum, 31, [7, 14])
assert result is False

tree = Tree.build(ctx.get().called_with_output_values)
ctx.reset(token)
tree.render(tree.root).to_image()

# Dynamic Programming

## Rod Cutting

- The recursive solution to Cut-Rod has a runtime of $\Theta(2^n)$. The algorithm calls itself over and
over again with the same parameters, resulting in the same subproblems being computed several
timectx.
- The dynamic programming solution for Cut-Rod runs in time $O(n^2)$ for a rod of length $n$. It
uses $O(n)$ space to store the values to the subproblemctx.

In [8]:
from src.dp.record import DpList
from IPython.display import display, Markdown

def dynamic_cut_rod(n: int, p: List[int]):
    assert len(p) >= n + 1

    r = DpList([0] * (n+1))
    s = [1] * (n+1)

    r.start("initialization",r=r[:], s=s[:]).stop()

    for j in range(1, n + 1):
        for k in range(1, j + 1):
            if p[k] + r[j-k] > r[j]:
                r[j] = p[k] + r[j-k]
                s[j] = k
                r.start("dp_fill", j=j, k=k, r=r[:], s=s[:]).stop()
    j = n
    cuts = []
    while j > 0:
        cuts.append(s[j])
        j -= s[j]
        r.start("reconstruction", cuts=cuts[:], j=j).stop()

    return r[n], cuts, r

# Have to add a 0 at the beginning
prices = [0,2,2,7,9,7,2]

value, cuts, dp = dynamic_cut_rod(6, prices)
# assert value == 10 and cuts == [2, 2]
# assert dynamic_cut_rod(3, prices)[0] == 7
# assert dynamic_cut_rod(2, prices)[0] == 5
# assert dynamic_cut_rod(1, prices)[0] == 2
# assert dynamic_cut_rod(0, prices)[0] == 0


display(Markdown(f"""
### Steps
#### Initialization

{dp.to_markdown("initialization")}

#### DP Fill

{dp.to_markdown("dp_fill")}

#### Reconstruction

{dp.to_markdown("reconstruction")}
"""))


### Steps
#### Initialization

| r                     | s                     |
|-----------------------|-----------------------|
| [0, 0, 0, 0, 0, 0, 0] | [1, 1, 1, 1, 1, 1, 1] |

#### DP Fill

|   j |   k | r                       | s                     |
|-----|-----|-------------------------|-----------------------|
|   1 |   1 | [0, 2, 0, 0, 0, 0, 0]   | [1, 1, 1, 1, 1, 1, 1] |
|   2 |   1 | [0, 2, 4, 0, 0, 0, 0]   | [1, 1, 1, 1, 1, 1, 1] |
|   3 |   1 | [0, 2, 4, 6, 0, 0, 0]   | [1, 1, 1, 1, 1, 1, 1] |
|   3 |   3 | [0, 2, 4, 7, 0, 0, 0]   | [1, 1, 1, 3, 1, 1, 1] |
|   4 |   1 | [0, 2, 4, 7, 9, 0, 0]   | [1, 1, 1, 3, 1, 1, 1] |
|   5 |   1 | [0, 2, 4, 7, 9, 11, 0]  | [1, 1, 1, 3, 1, 1, 1] |
|   6 |   1 | [0, 2, 4, 7, 9, 11, 13] | [1, 1, 1, 3, 1, 1, 1] |
|   6 |   3 | [0, 2, 4, 7, 9, 11, 14] | [1, 1, 1, 3, 1, 1, 3] |

#### Reconstruction

| cuts   |   j |
|--------|-----|
| [3]    |   3 |
| [3, 3] |   0 |


## Longest palindromic substring

This algorithm carries out a constant number of operations per cell of the table. Since there
are $\Theta(n^2)$ entries in the table, the algorithm runs in time $\Theta(n^2)$. This is a substantial improvement of the
$\Theta(n^3)$ brute-force algorithm.

In [9]:
from src.dp.record import DpList
from IPython.display import display, Markdown

def longest_palindrome_substring(s: str):
    n = len(s)
    max_str = s

    if n <= 1:
        return s

    dp = DpList([[False] * n for _ in range(n)])
    max_length = 1

    for i in range(n):
        dp[i][i] = True

    dp.start("initialization", dp=dp.deepcopy(), max_length=max_length).stop()

    for i in range(n-1):
        if s[i] == s[i+1]:
            dp[i][i+1] = True
            max_length = 2
            max_str = s[i:i+2]
            dp.start("dp_fill_1", i=i, dp=dp.deepcopy(), max_length=max_length).stop()

    for k in range(3, n + 1):
        for i in range(n - k + 1):
            j = i + k - 1
            if dp[i+1][j-1] and s[i] == s[j]:
                dp[i][j] = True
                max_length = k
                max_str = s[i:j+1]
                dp.start("dp_fill_2", k=k, i=i, j=j, s_i=s[i], s_j=s[j], dp=dp.deepcopy(), max_length=max_length).stop()

    return max_length, max_str, dp

assert longest_palindrome_substring("dabzbae")[0] == 5
assert longest_palindrome_substring("dabzzbae")[0] == 6
assert longest_palindrome_substring("dabzqbae")[0] == 1
assert longest_palindrome_substring("dd")[0] == 2

max_length, word, dp = longest_palindrome_substring("cccccabba")

display(Markdown(f"""
### Steps
#### Initialization

{dp.to_markdown("initialization")}

#### Fill Same Characters Next to Each Other

{dp.to_markdown("dp_fill_1")}

#### Final Fill

{dp.to_markdown("dp_fill_2")}
"""))


### Steps
#### Initialization

| dp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |   max_length |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------|
| $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            1 |

#### Fill Same Characters Next to Each Other

|   i | dp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |   max_length |
|-----|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------|
|   0 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            2 |
|   1 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            2 |
|   2 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            2 |
|   3 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            2 |
|   6 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            2 |

#### Final Fill

|   k |   i |   j | s_i   | s_j   | dp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |   max_length |
|-----|-----|-----|-------|-------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------|
|   3 |   0 |   2 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            3 |
|   3 |   1 |   3 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            3 |
|   3 |   2 |   4 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            3 |
|   4 |   0 |   3 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            4 |
|   4 |   1 |   4 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            4 |
|   4 |   5 |   8 | a     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            4 |
|   5 |   0 |   4 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   1 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |            5 |


## Longest Palindromic Subsequence

This algorithm carries out a constant number of operations per cell of the table. Since there
are $Θ(n^2)$ entries in the table, the algorithm runs in time $Θ(n^2)$. This is a substantial improvement of the
$Θ(2^n)$ brute-force algorithm.

In [10]:
from src.dp.record import DpList
from src.display import Display

def longest_palindrome_subsequence(s: str):
    n = len(s)
    dp = DpList([[0] * len(s) for _ in range(n)])

    for i in range(n):
        dp[i][i] = 1

    dp.start("initialization", dp=dp.deepcopy()).stop()

    for i in range(n-1):
        value = 2 if s[i] == s[i+1] else 1
        dp[i][i+1] = value
        dp.start("dp_fill_1", i=i, dp=dp.deepcopy()).stop()


    for k in range(3, n + 1):
        for i in range(n - k + 1):
            j = i + k - 1

            if s[i] == s[j]:
                dp[i][j] = 2 + dp[i+1][j-1]
            else:
                dp[i][j] = max(dp[i+1][j], dp[i][j-1])

            dp.start("dp_fill_2", k=k, i=i, j=j, s_i=s[i], s_j=s[j], dp=dp.deepcopy()).stop()

    return dp[0][n-1], dp

dp = longest_palindrome_subsequence("cccccabba")[1]

assert longest_palindrome_subsequence("dabzzobae")[0] == 6
assert longest_palindrome_subsequence("dabzobae")[0] == 5
assert longest_palindrome_subsequence("dabbae")[0] == 4

Display.md(f"""
### Steps
#### Initialization

{dp.to_markdown("initialization")}

#### Fill Same Characters Next to Each Other

{dp.to_markdown("dp_fill_1")}

#### Final Fill

{dp.to_markdown("dp_fill_2")}
""")


### Steps
#### Initialization

| dp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |

#### Fill Same Characters Next to Each Other

|   i | dp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|-----|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   0 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   1 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   2 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   5 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   6 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   7 | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |

#### Final Fill

|   k |   i |   j | s_i   | s_j   | dp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|-----|-----|-----|-------|-------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|   3 |   0 |   2 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 |   1 |   3 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 |   2 |   4 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 |   3 |   5 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 |   4 |   6 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 |   5 |   7 | a     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   3 |   6 |   8 | b     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 |   0 |   3 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 |   1 |   4 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   0 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 |   2 |   5 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   0 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 |   3 |   6 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   0 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 |   4 |   7 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   4 |   5 |   8 | a     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   5 |   0 |   4 | c     | c     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   0 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   5 |   1 |   5 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   0 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   5 |   2 |   6 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   0 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   5 |   3 |   7 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   0 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   5 |   4 |   8 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   0 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   6 |   0 |   5 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   0 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   6 |   1 |   6 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   0 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   6 |   2 |   7 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   0 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   6 |   3 |   8 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   0 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   7 |   0 |   6 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   5 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   0 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   7 |   1 |   7 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   5 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   4 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   0 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   7 |   2 |   8 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   5 &   0 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   4 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   4 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   8 |   0 |   7 | c     | b     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   5 &   5 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   4 &   0 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   4 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   8 |   1 |   8 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   5 &   5 &   0 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   4 &   4 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   4 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |
|   9 |   0 |   8 | c     | a     | $$\begin{array}{rrrrrrrrr} \hline    0 &   1 &   2 &   3 &   4 &   5 &   6 &   7 &   8 \\ \hline    1 &   2 &   3 &   4 &   5 &   5 &   5 &   5 &   5 \\    0 &   1 &   2 &   3 &   4 &   4 &   4 &   4 &   4 \\    0 &   0 &   1 &   2 &   3 &   3 &   3 &   3 &   4 \\    0 &   0 &   0 &   1 &   2 &   2 &   2 &   2 &   4 \\    0 &   0 &   0 &   0 &   1 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   1 &   1 &   2 &   4 \\    0 &   0 &   0 &   0 &   0 &   0 &   1 &   2 &   2 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &   1 \\    0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 \\ \hline \end{array}$$ |


In [11]:
from src.dp.record import DpList
from src.display import Display
from typing import List

INF = float("inf")

def min_battery_stops(b: List[int]):
    n = len(b)  # Since we start at station 0
    dp = DpList([0] + ([INF] * (n - 1))) # No stops needed at the start, every other station is inf

    dp.start("initialization", dp=dp[:], b=b).stop()
    
    for i in range(1, n):
        for j in range(0, i):
            if j + b[j] >= i:  # Check if we can reach station i from station j
                dp[i] = min(dp[i], dp[j] + 1)
                dp.start("dp_fill_1", dp=dp[:], i=i, j=j, b=b).stop()
    
    min_steps = dp[n-1]

    if min_steps == INF:
        return INF, None, dp  # If we can't reach the end, return INF
    else:

        stations = []
        current_length = min_steps
        for i in range(n - 1, -1, -1):
            if dp[i] == current_length:
                stations.append(b[i])
                current_length -= 1
                dp.start("reconstruction", dp=dp[:], i=i, b=b, stations=stations[:]).stop()
        
        # The stations is constructed in reverse, so we reverse it
        stations.reverse()

        return min_steps, stations, dp

b = [2, 0, 3, 4, 8, 1, 9]
stops, stations, dp = min_battery_stops(b)
print("The minimum number of stops:", stops)
print("The stations stopped at:", stations)

Display.md(f"""
### Steps
#### Initialization

{dp.to_markdown("initialization")}

#### DP Fill

{dp.to_markdown("dp_fill_1")}

#### Reconstruction

{dp.to_markdown("reconstruction")}
""")

The minimum number of stops: 3
The stations stopped at: [2, 3, 1, 9]



### Steps
#### Initialization

| dp                                | b                     |
|-----------------------------------|-----------------------|
| [0, inf, inf, inf, inf, inf, inf] | [2, 0, 3, 4, 8, 1, 9] |

#### DP Fill

| dp                              |   i |   j | b                     |
|---------------------------------|-----|-----|-----------------------|
| [0, 1, inf, inf, inf, inf, inf] |   1 |   0 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, inf, inf, inf, inf]   |   2 |   0 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, inf, inf, inf]     |   3 |   2 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, inf, inf]       |   4 |   2 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, inf, inf]       |   4 |   3 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, 2, inf]         |   5 |   2 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, 2, inf]         |   5 |   3 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, 2, inf]         |   5 |   4 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, 2, 3]           |   6 |   3 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, 2, 3]           |   6 |   4 | [2, 0, 3, 4, 8, 1, 9] |
| [0, 1, 1, 2, 2, 2, 3]           |   6 |   5 | [2, 0, 3, 4, 8, 1, 9] |

#### Reconstruction

| dp                    |   i | b                     | stations     |
|-----------------------|-----|-----------------------|--------------|
| [0, 1, 1, 2, 2, 2, 3] |   6 | [2, 0, 3, 4, 8, 1, 9] | [9]          |
| [0, 1, 1, 2, 2, 2, 3] |   5 | [2, 0, 3, 4, 8, 1, 9] | [9, 1]       |
| [0, 1, 1, 2, 2, 2, 3] |   2 | [2, 0, 3, 4, 8, 1, 9] | [9, 1, 3]    |
| [0, 1, 1, 2, 2, 2, 3] |   0 | [2, 0, 3, 4, 8, 1, 9] | [9, 1, 3, 2] |
